In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

## binarize

In [ ]:
series = pd.Series([0, 2, 1, None, np.nan, "a"], name="Name")

kraft.binarize(series)

## cast_builtin

In [ ]:
for object_ in (
    "False",
    "True",
    "0",
    "1.0",
    None,
):

    output = kraft.cast_builtin(object_)

    print(output, type(output))

## clip_by_standard_deviation

In [ ]:
array = np.arange(-2, 3)

for standard_deviation in (-1, 0, 1, 2):

    print(kraft.clip_by_standard_deviation(array, standard_deviation))

## _

In [ ]:
point_x_dimension = np.random.random_sample(size=(100, 100))

block_size = int(point_x_dimension.shape[0] * 0.2)

point_x_dimension[:block_size] *= 2

point_x_dimension[-block_size:] *= 1 / 2

point_x_dimension = pd.DataFrame(
    point_x_dimension,
    index=("Index{}".format(i) for i in range(point_x_dimension.shape[0])),
    columns=("Column{}".format(i) for i in range(point_x_dimension.shape[1])),
)

## cluster_hierarchical_clusterings

In [ ]:
kraft.cluster_hierarchical_clusterings(point_x_dimension, 0, 3)

## cluster_matrix_factorization_clusterings

In [ ]:
kraft.cluster_matrix_factorization_clusterings(point_x_dimension, 3)

## cluster

In [ ]:
kraft.cluster(point_x_dimension.values, 3)

## compute_bandwidth

In [ ]:
for power in range(1, 10):

    array = np.random.normal(size=2 ** power)

    print(array.size, kraft.compute_bandwidth(array))

## compute_coclustering_distance

In [ ]:
clustering_x_point = np.array(
    [[1, 1, np.nan, np.nan], [0, 0, 1, np.nan], [1, 0, np.nan, 1],]
)

kraft.compute_coclustering_distance(clustering_x_point)

## compute_margin_of_error

In [ ]:
for function in (np.random.random_sample, np.random.normal):

    print(function.__name__)

    for power in range(4):

        array = function(size=10 ** power)

        print(array.size, kraft.compute_margin_of_error(array))

## _

In [ ]:
random_values = np.arange(10)

values = np.asarray([0, 1, 8, 9])

## compute_p_value

In [ ]:
for value in values:

    print(value)

    for direction in ("<", ">"):

        print(direction)

        print(kraft.compute_p_value(value, random_values, direction))

## compute_p_values_and_q_values

In [ ]:
for direction in ("<", ">", "<>"):

    print(kraft.compute_p_values_and_q_values(values, random_values, direction))

## compute_set_enrichment

In [ ]:
n_element = 10

element_score = pd.Series(
    np.arange(n_element),
    index=pd.Index(("Element{}".format(i) for i in range(n_element)), name="Element"),
    name="Score",
)

for set_elements in (element_score.index[:2], element_score.index[-2:]):

    kraft.compute_set_enrichment(element_score, set_elements, plot=True)

## _

In [ ]:
point_x_dimensions = (
    np.asarray([[0], [1], [2],]),
    np.asarray([[0, 0], [1, 2], [2, 4],]),
    np.asarray([[0, 0, 0], [1, 2, 4], [2, 4, 8],]),
)

## estimate_density

In [ ]:
for point_x_dimension in point_x_dimensions:

    kraft.estimate_density(point_x_dimension)

## estimate_pdf

In [ ]:
for point_x_dimension in point_x_dimensions:

    kraft.estimate_pdf(point_x_dimension)

## estimate_posterior_pdf

In [ ]:
for point_x_dimension in point_x_dimensions:

    kraft.estimate_posterior_pdf(point_x_dimension)

## _

In [ ]:
a = 10

b = 20

v = np.random.normal(size=(a, b))

vs = (v, v * 10, v * 100)

r = 3

n_iteration = 10

for v in vs:

    for mode in ("ws", "hs"):

        ws, hs, errors = kraft.factorize_matrices(
            (v,), r, mode, n_iteration=n_iteration
        )

        kraft.plot_matrix_factorization(ws, hs, errors=errors)